# PricePredict LSTM Optimization

** Note: The problem with using jupiter notebook to run this code is that there is no way to turn off the output
   of the model summary during training. This is a problem because the output is too large and it bloats jupiter's
   memory. The code works fine in a python script. **
     
Using bayesian optimization to optimize the hyperparameters of the LSTM model.
- Do hidden layers improve the model?

- Latest test that results in the best trend prediction...

## Test Parameters
```
pp.bayesian_optimization(X, y,
                         opt_max_init=100,
                         opt_max_iter=20,
                         pb_lstm_units=(32, 256),
                         pb_lstm_dropout=(0.1, 0.5),
                         pb_adam_learning_rate=(0.001, 0.1),
                         pb_epochs=(100, 300),
                         pb_batch_size=(1, 1024),
                         pb_hidden_layers=(1, 4),
                         pb_hidden_layer_units_1=(16, 256),
                         pb_hidden_layer_units_2=(32, 256),
                         pb_hidden_layer_units_3=(64, 256),
                         pb_hidden_layer_units_4=(128, 256))  
 ```

## Best Hyperparameters 
```
{'adam_learning_rate': 0.007133047958571564,
 'batch_size': 778.4103564949222,
 'epochs': 207.8240175819626,
 'hidden_layer_units_1': 59.5543770288849,
 'hidden_layer_units_2': 33.32458485892822,
 'hidden_layer_units_3': 74.53889954301027,
 'hidden_layer_units_4': 186.98073315100643,
 'hidden_layers': 1.2787028637916622,
 'lstm_dropout': 0.1277210415797052,
 'lstm_units': 240.4579822805544
}
```
 
## Good Trend Prediction 62% tren prediction
```
{'actual_vs_pred': {'mean_trading_range': 0.0,
                    'mean_delta': 0.0,
                    'corr_day_cnt': 639,
                    'corr_days': 639,
                    'uncorr_days': 387,
                    'pct_corr': 62.2807,
                    'pct_uncorr': 37.7193},
 'actual_vs_adj_pred': {'mean_trading_range': 2.07,
                        'mean_delta': 1.42,
                        'corr_days': 639,
                        'uncorr_days': 387,
                        'pct_corr': 62.2807,
                        'pct_uncorr': 37.7193}
}
```


In [ ]:
%matplotlib inline

import logging
import os
import tqdm
from IPython.display import clear_output
from datetime import datetime
from pricepredict import PricePredict

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Create an instance of the price prediction object
pp = PricePredict(model_dir='../models/',
                  chart_dir='../charts/', preds_dir='../predictions/',
                  verbose=False, logger=logger, log_level=logging.ERROR,
                  keras_verbosity=0)

# Load data from Yahoo Finance
ticker = "IBM"
start_date = "2020-01-01"
end_date = "2023-12-31"
# Load and prep data from Yahoo Finance
pp.ticker = ticker
X, y = pp._fetch_n_prep(pp.ticker, start_date, end_date, train_split=0.8)
# Train the model
model, y_pred, mse = pp.train_model(X, y)

# Perform Bayesian optimization
# - Test with all Parameters and 1 2 and 3 hidden layers
pp.model = None
pp.bayes_opt_hypers = None
pp.bayesian_optimization(X, y,
                         opt_max_init=5,
                         opt_max_iter=10,
                         pb_lstm_units=(32, 256),
                         pb_lstm_dropout=(0.1, 0.5),
                         pb_adam_learning_rate=(0.001, 0.1),
                         pb_epochs=(100, 300),
                         pb_batch_size=(1, 1024),
                         pb_hidden_layers=(1, 4),
                         pb_hidden_layer_units_1=(16, 256),
                         pb_hidden_layer_units_2=(32, 256),
                         pb_hidden_layer_units_3=(64, 256),
                         pb_hidden_layer_units_4=(128, 256))

clear_output()

time = datetime.now()
time_str = time.strftime('%Y-%m-%d %H:%M:%S')
title = f"test_bayes_opt all params: {ticker} --  Period {pp.period}  {time_str}"
pred_close = pp.pred[:, 1]
pred_high = pp.pred[:, 2]
pred_low = pp.pred[:, 3]
pp.plot_pred_results(pp.target_close, pp.target_high, pp.target_low,
                     pred_close, pred_high, pred_low, title=title)
